In [1]:
import pandas as pd
# %pip install tensorflow[and-cuda]
import os

dir_notebook = os.getcwd()
print(dir_notebook)
ruta_trabajo = '../../../'

# Cambiar el directorio de trabajo
os.chdir(ruta_trabajo)
print("Directorio de trabajo actual:", os.getcwd())

/home/david/proyectos4/repositorios/Proyectos-SGBA1/Pruebas/Modelos/precio_peninsula
Directorio de trabajo actual: /home/david/proyectos4/repositorios/Proyectos-SGBA1


In [2]:
df_precio = pd.read_parquet('data/processed/datos_precio/clima_precio_merged.parquet')

In [3]:
df_precio.head()

,timestamp,€/kwh,fecha,indicativo_3196,altitud_3196,tmed_3196,tmin_3196,tmax_3196,velmedia_3196,racha_3196,...,horaPresMax_cos_8175,horaPresMin_varias_8175,horaPresMin_minutos_8175,horaPresMin_sin_8175,horaPresMin_cos_8175,prec_inapreciable_8175,prec_valor_8175,prec_log_8175,dir_sin_8175,dir_cos_8175
0,2014-11-03 14:00:00,0.0,2014-11-03,3196.0,690.0,11.5,8.0,15.0,6.1,17.5,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.984808,-0.173648
1,2014-11-03 15:00:00,0.0,2014-11-03,3196.0,690.0,11.5,8.0,15.0,6.1,17.5,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.984808,-0.173648
2,2014-11-03 16:00:00,0.0,2014-11-03,3196.0,690.0,11.5,8.0,15.0,6.1,17.5,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.984808,-0.173648
3,2014-11-03 17:00:00,0.0,2014-11-03,3196.0,690.0,11.5,8.0,15.0,6.1,17.5,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.984808,-0.173648
4,2014-11-03 18:00:00,0.0,2014-11-03,3196.0,690.0,11.5,8.0,15.0,6.1,17.5,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.984808,-0.173648


In [4]:
def prepare_data(df):
    # Create base prophet dataframe with datetime and target
    base_cols = {'timestamp': 'ds', '€/kwh': 'y'}
    prophet_df = df.reset_index()[list(base_cols.keys())].rename(columns=base_cols)
    
    # Get additional numeric columns (excluding timestamp and target)
    numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
    additional_cols = [col for col in numeric_columns 
                      if col != '€/kwh' and 'timestamp' not in col and 'fecha' not in col]
    
    # Combine all columns efficiently using concat
    if additional_cols:
        prophet_df = pd.concat([
            prophet_df,
            df[additional_cols]
        ], axis=1)
    
    # Convert any potential NaN values to 0
    return prophet_df.fillna(0)

In [5]:
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
from sklearn.model_selection import TimeSeriesSplit

def train_evaluate_prophet(df, params, n_splits=5):
    """
    Trains and evaluates a Prophet model using TimeSeriesSplit for validation.

    Args:
        df (pd.DataFrame): Input DataFrame with 'ds' (datetime) and 'y' (target) columns.
        params (dict): Parameters for the Prophet model.
        n_splits (int): Number of splits for TimeSeriesSplit.

    Returns:
        tuple: (model, metrics, forecast)
               model: Trained Prophet model.
               metrics: Dictionary of evaluation metrics.
               forecast: DataFrame with forecast values.
    """
    
    tscv = TimeSeriesSplit(n_splits=n_splits)
    metrics_list = []
    models = []
    forecasts = []

    for train_index, test_index in tscv.split(df):
        train_df = df.iloc[train_index]
        test_df = df.iloc[test_index]

        # Initialize and train Prophet model
        model = Prophet(**params)

        # Add every column as regressor except 'ds' and 'y'
        for col in train_df.columns:
            if col not in ['ds', 'y']:
                model.add_regressor(col)

        
        model.fit(train_df)
        
        

        # Make predictions
        future = model.make_future_dataframe(periods=len(test_df))
        forecast = model.predict(future)
        
        # Calculate metrics
        y_true = test_df['y'].values
        y_pred = forecast.tail(len(test_df))['yhat'].values
        
        metrics = {
            'mae': mean_absolute_error(y_true, y_pred),
            'mse': mean_squared_error(y_true, y_pred),
            'rmse': np.sqrt(mean_squared_error(y_true, y_pred)),
            'r2': r2_score(y_true, y_pred)
        }
        metrics_list.append(metrics)
        models.append(model)
        forecasts.append(forecast)
    
    # Average the metrics across all splits
    avg_metrics = {}
    for metric in metrics_list[0].keys():
        avg_metrics[metric] = np.mean([m[metric] for m in metrics_list])

    # You might want to train a final model on the entire dataset
    final_model = Prophet(**params)
    final_model.fit(df)
    future = final_model.make_future_dataframe(periods=0)  # No future periods for the final model
    final_forecast = final_model.predict(future)

    return final_model, avg_metrics, final_forecast

/home/david/proyectos4/p4_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
!pwd

/home/david/proyectos4/repositorios/Proyectos-SGBA1


In [ ]:
# Prepare data for Prophet
prophet_df = prepare_data(df_precio)

# Define parameters to try
prophet_params = {
    'yearly_seasonality': True,
    'weekly_seasonality': True,
    'daily_seasonality': True,
}

model, metrics, forecast = train_evaluate_prophet(prophet_df, prophet_params)

print(metrics)

19:19:20 - cmdstanpy - INFO - Chain [1] start processing
19:19:21 - cmdstanpy - INFO - Chain [1] done processing
19:19:21 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
19:19:26 - cmdstanpy - INFO - Chain [1] start processing


KeyboardInterrupt: 

In [2]:
import dagshub
dagshub.init(repo_owner='auditoria.SGBA1', repo_name='Proyectos-SGBA1', mlflow=True)

Accessing as auditoria.SGBA1

Initialized MLflow to track repo "auditoria.SGBA1/Proyectos-SGBA1"

Repository auditoria.SGBA1/Proyectos-SGBA1 initialized!

In [ ]:
import mlflow

experiment_prefix = 'prediccion_precio_energia'

experiment_name = f'{experiment_prefix}_prophet'
mlflow.set_experiment(experiment_name)

In [ ]:
prophet_params = {
    'yearly_seasonality': True,
    'weekly_seasonality': True,
    'daily_seasonality': True,
}

prophet_df = prepare_data(df_precio)

with mlflow.start_run(run_name='prophet_base'):
    # Log parameters
    mlflow.log_params(prophet_params)

    model, metrics, forecast = train_evaluate_prophet(prophet_df, prophet_params)

    # Log metrics
    mlflow.log_metrics(metrics)

    # Log model
    mlflow.log_artifact('prophet_base_model.pkl')
    # mlflow.log_artifact('prophet_base_forecast.pkl')

    # Log feature importance
    fig1 = model.plot(forecast)
    fig2 = model.plot_components(forecast)
    # mlflow.log_artifact('feature_importance.png')
